In [ ]:
from sklearn.datasets import load_iris, load_breast_cancer
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score

from sklearn.preprocessing import LabelEncoder

import pandas as pd
import numpy  as np

In [ ]:
# 1. 데이터 로드 및 확인
titanic_train=pd.read_csv('/content/sample_data/titanic_train.csv')

In [ ]:
titanic_train

In [ ]:
# 2. 데이터 정보 확인
titanic_train.info()

In [ ]:
# [간단한 방법이 있음..]

# titanic_train['Age'].fillna(titanic_train['Age'].mean(),inplace=True)
# titanic_train['Cabin'].fillna('N',inplace=True)
# titanic_train['Embarked'].fillna('N',inplace=True)

# print('데이터 세트 Null 값 갯수 ',titanic_train.isnull().sum())

In [ ]:
# 3. missingno 패키지를 이용한 결측값을 가지는 컬럼 확인 및 처리
# -  age는 평균으로, cabin 'N', embarked 'N' 으로 대체
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

import missingno as msno

In [ ]:
print(titanic_train.isnull().sum())

In [ ]:
msno.matrix(titanic_train) #결측데이터를 시각화 결측치는 흰색으로 표시됨 
# 오른쪽 라인은 해당 로우의 결측값이 없는 곳을 알려줌..?
plt.show()

In [ ]:
from sklearn.impute import SimpleImputer

In [ ]:
#  age를 평균 값으로 대체
imputer=SimpleImputer(strategy='mean')

In [ ]:
## fit(학습시키고),transform(값을 대체) 해당 값을 변수에 저장
result=imputer.fit_transform(titanic_train[['Age']])

In [ ]:
## 변경 후 저장한 값을 대체
titanic_train['Age']=result

In [ ]:
print(titanic_train.isnull().sum())

In [ ]:
# 여러개의 칼럼에 결측치 변환

In [ ]:
imputer=SimpleImputer(strategy='constant',fill_value='N')

In [ ]:
#fit(학습시키고),transform(값을 대체하고) 해당 값을 변수에 저장
resultce=imputer.fit_transform(titanic_train[['Cabin','Embarked']])

In [ ]:
## 변경 후 저장한 값을 대체
titanic_train[['Cabin','Embarked']]=resultce

In [ ]:
# 확인
print(titanic_train.isnull().sum())

In [ ]:
# 4. age , cabin , embarked 빈도확인

In [ ]:
print(titanic_train['Age'].value_counts())
print('*'*50)
print(titanic_train['Cabin'].value_counts())
print('*'*50)
print(titanic_train['Embarked'].value_counts())

In [ ]:
#5. 성별에 따른 생존여부 확인 및 barplot를 이용한 시각화
#- 수업시간에 배운 시각화를 이용하여 다양한 정보를 탐색하세요
import seaborn as sns

sexsurvived=titanic_train.groupby('Sex')[['Survived']].mean()
sexsurvived

sns.barplot(x='Sex',y='Survived',data=titanic_train)

In [ ]:
# Pclass 선실 등급에 따른 생존 Survived
sns.barplot(x='Pclass',y='Survived',data=titanic_train)

In [ ]:
# 6. age , cabin , embarked 에 대한 라베인코딩
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
titanic_train['Sex']=le.fit_transform(titanic_train['Sex'])
titanic_train['Cabin']=le.fit_transform(titanic_train['Cabin'])
titanic_train['Embarked']=le.fit_transform(titanic_train['Embarked'])
# 변환된 Label classes 확인
le.classes_
# 변환한 뒤 inverse_trainsform으로 다시 역변환 
# le.inverse_transform(result)

In [ ]:
# 입력 age에 따라 구분값을 반환하는 함수 설정. DataFrame의 apply lambda식에 사용. 
def get_category(age):
    cat = ''
    if age <= -1: cat = 'Unknown'
    elif age <= 5: cat = 'Baby'
    elif age <= 12: cat = 'Child'
    elif age <= 18: cat = 'Teenager'
    elif age <= 25: cat = 'Student'
    elif age <= 35: cat = 'Young Adult'
    elif age <= 60: cat = 'Adult'
    else : cat = 'Elderly'
    
    return cat

# 막대그래프의 크기 figure를 더 크게 설정 
plt.figure(figsize=(10,6))

#X축의 값을 순차적으로 표시하기 위한 설정 
group_names = ['Unknown', 'Baby', 'Child', 'Teenager', 'Student', 'Young Adult', 'Adult', 'Elderly']

# lambda 식에 위에서 생성한 get_category( ) 함수를 반환값으로 지정. 
# get_category(X)는 입력값으로 'Age' 컬럼값을 받아서 해당하는 cat 반환
titanic_train['Age_cat'] = titanic_train['Age'].apply(lambda x : get_category(x))
sns.barplot(x='Age_cat', y = 'Survived', hue='Sex', data=titanic_train, order=group_names)
titanic_train.drop('Age_cat', axis=1, inplace=True)

In [ ]:
# [ML학습]
# 1. 원본 데이터를 재로딩 하고, feature데이터 셋과 Label 데이터 셋 추출.
#titanic_train=pd.read_csv('/content/sample_data/titanic_train.csv')
#titanic_train.head()

In [ ]:
titanic=titanic_train.drop(['PassengerId','Name','Ticket'], axis='columns',inplace=False)
titanic

In [ ]:
y_titanic_label=titanic['Survived']
X_titanic_features=titanic.drop('Survived',axis=1)

In [ ]:
from sklearn.model_selection import GridSearchCV, train_test_split
X_train,X_test,y_train,y_test=train_test_split(X_titanic_features,
                                               y_titanic_label,
                                               test_size=0.2) # random_state=11 예제를 수행할때마다 같은 결과 나오게함
# [ML학습] ML알고리즘인 결정 트리, 랜덤 포레스트, 로지스틱 회귀를 이용해 생존자 예측할것
# train_test_fit()으로 분리시킨 학습 데이터와 테스트 데이터를 기반으로
# 머신러닝 학습하고(fit), 예측(predict)하고 예측 성능평가는 (accuracy_score())사용

In [ ]:
from sklearn.tree import DecisionTreeClassifier #결정트리

# 3. 
# DecisionTreeClassifier를 이용한 학습, 예측 및 정확도 확인
dt_clf=DecisionTreeClassifier(random_state=11)

# DecisionTreeClassifier 학습/예측/평가
dt_clf.fit(X_train,y_train)
dt_pred=dt_clf.predict(X_test)
print('DecisionTreeClassifier 정확도:{0:.4f}'.format(accuracy_score(y_test,dt_pred)))
print("*"*50)

# RandomForest를 이용한 학습, 예측 및 정확도 확인
from sklearn.ensemble import RandomForestClassifier
rf_clf=RandomForestClassifier(random_state=11)

# RandomForestClassifier 학습/예측/평가
rf_clf.fit(X_train,y_train)
rf_pred=rf_clf.predict(X_test)
print('RandomForestClassifier 정확도:{0:.4f}'.format(accuracy_score(y_test,rf_pred)))
print("*"*50)

# LogisticRegression을 이용한 학습, 예측 및 정확도 확인
from sklearn.linear_model import LogisticRegression
lr_clf=LogisticRegression(random_state=11)

# LogisticRegression 학습/예측/평가
lr_clf.fit(X_train,y_train)
lr_pred=lr_clf.predict(X_test)
print('LogisticRegression 정확도:{0:.4f}'.format(accuracy_score(y_test,lr_pred)))

In [ ]:
# 4. KFold=5 를 이용한 교차검증 및 평균정확도 확인
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier # 학습 모델 알고리즘 선택
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import pandas as pd
import numpy as np

In [ ]:
# Fold 교차검증
# Fold 교차검증은 데이터셋을 K개로 나눈 뒤 하나를 평가 데이터셋으로, 
# 나머지를 학습 데이터셋으로 하여 모델을 학습시키고 평가하는 방법


from sklearn.model_selection import KFold

def exec_kfold(clf, folds=5):
    # 폴드 세트를 5개인 KFold객체를 생성, 폴드 수만큼 예측결과 저장을 위한  리스트 객체 생성.
    kfold = KFold(n_splits=folds)
    scores = []
    
    # KFold 교차 검증 수행. 
    for iter_count , (train_index, test_index) in enumerate(kfold.split(X_titanic_features)):
        # X_titanic_df 데이터에서 교차 검증별로 학습과 검증 데이터를 가리키는 index 생성
        X_train, X_test = X_titanic_features.values[train_index], X_titanic_features.values[test_index]
        y_train, y_test = y_titanic_label.values[train_index], y_titanic_label.values[test_index]
        
        # Classifier 학습, 예측, 정확도 계산 
        clf.fit(X_train, y_train) 
        predictions = clf.predict(X_test)
        accuracy = accuracy_score(y_test, predictions)
        scores.append(accuracy)
        print("교차 검증 {0} 정확도: {1:.4f}".format(iter_count, accuracy))     
    
    # 5개 fold에서의 평균 정확도 계산. 
    mean_score = np.mean(scores)
    print("평균 정확도: {0:.4f}".format(mean_score)) 
# exec_kfold 호출
exec_kfold(dt_clf , folds=5) 

In [ ]:
# cross_val_score() 함수는 데이터셋을 K개로 나누고 K번 반복하면서 평가하는 작업을 처리해줌

from sklearn.model_selection import cross_val_score

scores=cross_val_score(dt_clf,X_titanic_features,y_titanic_label,cv=5)

for iter_count, accuracy in enumerate(scores):
    print("교차 검증{0} 정확도:{1:4f}".format(iter_count,accuracy))
print("평균 정확도 :{0:4f}.format(np.mean(scores)")

In [ ]:
from sklearn.model_selection import GridSearchCV

parameters={'max_depth':[2,3,5,10],'min_samples_split':[2,3,5],'min_samples_leaf':[1,5,8]}

grid_dclf=GridSearchCV(dt_clf,param_grid=parameters,scoring='accuracy',cv=5)
grid_dclf.fit(X_train,y_train)

print('GridSearchCV 최적 하이퍼 파라미터:',grid_dclf.best_params_)
print('GridSearchCV 최고 정확도:{0:4f}:',format(grid_dclf.best_score_))
best_dclf=grid_dclf.best_estimator_

# GridSearchCV의 최적 하이퍼 파라미터로 학습된 Estimator로 예측 및 평가 수행
dpredictions=best_dclf.predict(X_test)
accuracy=accuracy_score(y_test,dpredictions)
print('테스트 세트에서 DecisionTreeClassifier 정확도:{0:.4f}'.format(accuracy))